In [ ]:
import pandas as pd
import rasterio
import geopandas as gpd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

def extract_pixel_chips(gdf, raster, chip_size=10):
    half_chip = chip_size // 2
    values = []
    for point in gdf.geometry:
        row, col = raster.index(point.x, point.y)
        pixel_values = raster.read()[:, row-half_chip:row+half_chip, col-half_chip:col+half_chip]
        values.append(pixel_values)
    return np.array(values)

class HabitatCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(HabitatCNN, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2)
        self.fc = nn.Linear(32 * 5 * 5, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

def train_model(model, criterion, optimizer, X_train, y_train, X_test, y_test, num_epochs=100):
    train_losses, test_losses = [], []

    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

        model.eval()
        with torch.no_grad():
            output = model(X_test)
            loss = criterion(output, y_test)
            test_losses.append(loss.item())

        if (epoch + 1) % 10 == 0:
            print(f"Epoch: {epoch+1}/{num_epochs}, Train Loss: {train_losses[-1]}, Test Loss: {test_losses[-1]}")

    return train_losses, test_losses

csv_file = 'jaguar_locations.csv'
df = pd.read_csv(csv_file)
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

satellite_image = 'satellite_image.tif'
with rasterio.open(satellite_image) as src:
    image = src.read()
    profile = src.profile

X = extract_pixel_chips(gdf, src)
y = gdf['habitat'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
X_test, y_test = torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)

num_classes = len(np.unique(y))
input_channels = X.shape[1]
model = HabitatCNN(input_channels, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0


To save the classified result as a new GeoTIFF, you'll need to predict the habitat class for each 10x10 pixel chip in the satellite image and then merge the classified chips into a single raster. Here's how you can do that:

Create a function to split the entire satellite image into 10x10 pixel chips with a stride (step) equal to the chip size:

In [ ]:
def split_image_into_chips(image, chip_size=10):
    chips = []
    rows, cols = image.shape[1], image.shape[2]
    for row in range(0, rows, chip_size):
        for col in range(0, cols, chip_size):
            chip = image[:, row:row+chip_size, col:col+chip_size]
            chips.append(chip)
    return np.array(chips)


In [ ]:
# Split the entire satellite image into 10x10 pixel chips:
image_chips = split_image_into_chips(image)


In [ ]:
# Predict the habitat class for each chip:
model.eval()
with torch.no_grad():
    image_chips_tensor = torch.tensor(image_chips, dtype=torch.float32)
    predictions = model(image_chips_tensor).argmax(dim=1).numpy()


In [ ]:
# Reshape the predictions into the original image shape:
classified_image = predictions.reshape(image.shape[1] // 10, image.shape[2] // 10)


In [ ]:
# Save the classified image as a new GeoTIFF:
output_file = 'classified_jaguar_habitat.tif'

output_profile = profile.copy()
output_profile.update(
    dtype=rasterio.uint8,
    count=1,
    compress='lzw'
)

with rasterio.open(output_file, 'w', **output_profile) as dst:
    dst.write(classified_image.astype(rasterio.uint8), 1)


Please note that this method assumes that the input satellite image's dimensions are divisible by the chip size (10x10 pixels in this case). If the dimensions are not divisible, you might need to pad the image to make it divisible or handle the border cases separately.

Also, this method predicts the habitat class for each 10x10 pixel chip independently, without considering the spatial relationship between neighboring chips. If you need a more accurate classification that considers the spatial context, you can try using a sliding window approach or a Fully Convolutional Network (FCN) for semantic segmentation.